# 1. Word Embedding
Thực hiện chuyển đổi câu sang vector dùng Word2Vec. Tôi sẽ tạo một vector ngẫu nhiên cho từ UNK, từ UNK đại diện cho các từ hiếm gặp, không xuất hiện trong tập vocabulary

In [ ]:
import pandas as pd, numpy as np
from gensim.models import Word2Vec
from sklearn.preprocessing import OneHotEncoder

def get_wv(w):
    try:
        return w2v.wv[w]
    except KeyError:
        return w2v.wv["UNK"]

df = pd.read_csv("./dataset/DanhgiaSmartphone.csv")
sentences = df["comment"].values
words = [[word for word in sen.lower().split()] for sen in sentences]
words = [w + ['UNK'] for w in words]
n = len(sentences)
y = OneHotEncoder(sparse_output=False).fit_transform(df["label"].values.reshape(-1, 1))
# [001] pos
# [100] neg
# [010] neu
# Chuyen word sang vector
# w2v = Word2Vec(words, vector_size=10)
# w2v.save("test") 
w2v = Word2Vec.load("test")

X = []
for word in words:
    data = []
    for w in word:
        data.append(get_wv(w))
    X.append(data)
X = np.array(X, dtype=object)




# 2. Mô hình RNN

Hình này đại diện cho những gì xảy ra ở 1 step
<p align="center" style="background: white">
<img src="https://images.viblo.asia/4b1cc09d-99fa-422a-9bee-14908aace750.png" height="300">
</p>

B1: Chúng ta có $h_{t-1}$ là state của các step phía trước. Bắt đầu, ta tính $z_t = x_t @ W_{hx} + b_h$ như thường lệ.
Sau đó cộng $z_t$ với state của các step phía trước $z_t + (h_{t-1}@W_{hh})$. Tiếp tục đưa tất cả qua hàm kích hoạt $g_1$ để được state của step hiện tại $h_t = g_1(z_t + (h_{t-1}@W_{hh}))$.   
B2: Nếu đầu vào chưa kết thúc, quay lại B1  
B3: Đến đây, nếu đầu vào đã kết thúc (cuối câu) thì ta tính output $y_t = g_2(h_t @ W_{yh} + b_y)$. 

## Import library, create train and test data

In [8]:
%reload_ext autoreload
%autoreload 2
import re
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from sklearn.preprocessing import OneHotEncoder
from module.layers.Dense import Dense
from module.optimizer.Adam import Adam
from module.layers.RNN import RNN
from tensorflow import keras
from module.Sequential import Sequential


df = pd.read_csv("./dataset/DanhgiaSmartphone.csv")
data = df[["comment", "label"]].values
np.random.shuffle(data)
print(len(data)) # 113
train_data = data[:63]
test_data = data[:63]

113


# Build word embedding


In [9]:
# Hàm lấy word embedding cho 1 từ
def get_wv(w):
    try:
        return w2v.wv[w]
    except KeyError:
        return w2v.wv["UNK"]

# Hàm tạo word embedding cho 1 tập dữ liệu 
def word_embedding(sentences):
    sen_split = [[word for word in sen.lower().split()]for sen in sentences]
    x = []
    for s in sen_split:
        v = []
        for w in s:
            v.append(get_wv(w))
        x.append(v)
    x = keras.preprocessing.sequence.pad_sequences(x, padding="post", dtype="float32")
    return x

""" df = pd.read_csv("./dataset/UIT-ViSFD/Train.csv")
sen_embedding = df["comment"]

# Xóa dấu câu và số trong dataset dùng để xây dựng word embedding
sen_embedding_clean = []
for sen in sen_embedding:
    clean_sen = re.sub(r'[^\w\s]', '', sen)
    clean_sen = re.sub(r'\d', '', clean_sen)
    sen_embedding_clean.append(clean_sen)

sen_embedding_clean = [[word for word in sen.lower().split()] for sen in sen_embedding_clean]
# Chuyen word sang vector
w2v = Word2Vec(sen_embedding_clean, vector_size=10)
unk_vector = np.random.randn(10)
w2v.wv.add_vector("UNK", unk_vector)
w2v.save("test.model")  """
w2v = Word2Vec.load("test.model")

x_train = word_embedding(data[:, 0])
y_train = OneHotEncoder(sparse_output=False).fit_transform(data[:, 1].reshape(-1, 1))
print(x_train.shape)
print(y_train.shape)

(113, 6, 10)
(113, 3)


## Build model

In [13]:
md = Sequential()
md.add(RNN(32, active="relu"))
md.add(Dense(3, active="softmax"))
md.compile(optimizer=Adam(lr=0.003, beta1=0.9, beta2=0.99999))
md.fit(X=x_train, y=y_train, batch_size=6, epochs=100)

Epoch 0  [==========]  loss: 0.3623, accuracy 39.82%
Epoch 1  [==========]  loss: 0.3471, accuracy 40.71%
Epoch 2  [==========]  loss: 0.3303, accuracy 43.36%
Epoch 3  [==========]  loss: 0.3076, accuracy 48.67%
Epoch 4  [==========]  loss: 0.2914, accuracy 51.33%
Epoch 5  [==========]  loss: 0.2749, accuracy 52.21%
Epoch 6  [==========]  loss: 0.2613, accuracy 59.29%
Epoch 7  [==========]  loss: 0.2535, accuracy 69.91%
Epoch 8  [==========]  loss: 0.2361, accuracy 64.60%
Epoch 9  [==========]  loss: 0.2217, accuracy 66.37%
Epoch 10 [==========]  loss: 0.2054, accuracy 73.45%
Epoch 11 [==========]  loss: 0.2005, accuracy 76.99%
Epoch 12 [==========]  loss: 0.2089, accuracy 73.45%
Epoch 13 [==========]  loss: 0.1793, accuracy 79.65%
Epoch 14 [==========]  loss: 0.1789, accuracy 76.11%
Epoch 15 [==========]  loss: 0.1670, accuracy 80.53%
Epoch 16 [==========]  loss: 0.1900, accuracy 76.99%
Epoch 17 [==========]  loss: 0.1509, accuracy 82.30%
Epoch 18 [==========]  loss: 0.1479, accuracy 

## Test model's accuracy

In [14]:
x_test = word_embedding(test_data[:, 0])
y_test = OneHotEncoder(sparse_output=False).fit_transform(test_data[:, 1].reshape(-1, 1))
pre, score = md.evalute(x_test, y_test)
print(f"Test accuracy: {score:.2f}")

Test accuracy: 0.97
